# 36610 - Python and Unix for Bioinformaticians

## Project 6: Resistance to antibiotics

Details on how to read compressed files can be found [here](https://docs.python.org/3/library/gzip.html#gzip.decompress).

In [12]:
import gzip # import gzip to decompress .gz files

resistance_genes = open("data/resistance_genes.fsa", "r")
reads1 = gzip.open("data/Unknown3_raw_reads_1.txt.gz", "rt") # 'rt' -> read-in in text mode
reads2 = gzip.open("data/Unknown3_raw_reads_2.txt.gz", "rt")

In [24]:
# Open subsets to increase speed
ResFile = open("data/resistance_genes.fsa", "r")
reads1 = open("data/Unknown3_raw_reads_1.txt", "r")
#reads2 = open("data/Unknown3_raw_reads_2_sub.txt", "r")
outfile = open("data/results.txt", "w")

# define variables
header  = ""   # will contain the header for each FASTA sequence
sub_seq = ""   # i think this variable is not necessarily needed – used to store part of the FASTA sequence
seq     = ""   # used to store entire FASTA sequence
line_count = 3 # 
k_mer_length = 19
DNA_translation_table = str.maketrans("ACGT", "TGCA")

### resistance gene
    ### - k-mers
### -- read (use in; NO k-mer)
    ### for each k-mer in read
        ### count

for line in ResFile:
    line = line.rstrip()
    if line.startswith(">"):
        header = line # store header
        if sub_seq:
            seq += sub_seq
            print(header)
            #print(seq)
            
            # RES k-mer (list)
            # res_kmer = kmer_generator(seq, k_mer_length)
            
            # RES k-mer (set)
            res_kmer_set = k_mer_set_of_RES(seq, k_mer_length)
            #print(res_kmer_set)
            
            #print(header, file=outfile)
            
            # INFO FROM WEBSITE: There is no way of knowing if the reads come from one strand or the other - so try both.
            for line in reads1:
                line = line.rstrip()
                if (line_count % 4 == 0): # prints every 4th line (sequence line)
                    read_seq = line
                    
                    # also consider the reverse complement (other strand)
                    read_seq_rev_comp = reverse_complement(read_seq)
                    
                    # generate single read k-mer and see if in RES k-mer list
                    single_read_kmer = read_seq[0:0+k_mer_length]
                    single_read_kmer_rev_comp = read_seq_rev_comp[0:0+k_mer_length]
                    
                    # if the read kmer is found in the RES kmer set, we consider
                    # the entire read sequence
                    if (single_read_kmer in res_kmer_set or single_read_kmer_rev_comp in res_kmer_set):
                        print("Found read kmer in RES kmer.")
                        #print(single_read_kmer, single_read_kmer_rev_comp)
                line_count += 1
            seq = ""
    else:
        sub_seq = line
        seq += sub_seq

# here: process last FASTA entry as well

ResFile.close()
reads1.close()

>aac(6')-aph(2'')_1_KF652095 Aminoglycoside resistance:
>aadE_1_KF864551 Aminoglycoside resistance:
>aac(2')-Ia_1_L06156 Aminoglycoside resistance:
>aac(2')-Ib_1_U41471 Aminoglycoside resistance:
>aac(2')-Ic_1_U72714 Aminoglycoside resistance:
>aac(2')-Id_1_U72743 Aminoglycoside resistance:
>aac(2')-Ie_1_NC_011896 Aminoglycoside resistance:
>aac(3)-I_1_AJ877225 Aminoglycoside resistance:
>aac(3)-Ia_1_X15852 Aminoglycoside resistance:
>aac(3)-Ib_1_L06157 Aminoglycoside resistance:
>aac(3)-Ib-aac(6')-Ib_1_AF355189 Aminoglycoside resistance:
>aac(3)-Ic_1_AJ511268 Aminoglycoside resistance:
>aac(3)-Id_1_AB114632 Aminoglycoside resistance:
>aac(3)-If_1_AY884051 Aminoglycoside resistance:
>aac(3)-Ig_1_CP000282 Aminoglycoside resistance:
>aac(3)-Ih_1_CP000490 Aminoglycoside resistance:
>aac(3)-Ii_1_CP000356 Aminoglycoside resistance:
>aac(3)-IIa_1_X51534 Aminoglycoside resistance:
>aac(3)-IIb_1_M97172 Aminoglycoside resistance:
>aac(3)-IIc_1_X54723 Aminoglycoside resistance:
>aac(3)-IId_1_EU0

>blaCTX-M-86_1_FJ214369 Beta-lactam resistance:
>blaCTX-M-87_1_EU545409 Beta-lactam resistance:
>blaCTX-M-88_1_FJ873739 Beta-lactam resistance:
>blaCTX-M-89_1_FJ966096 Beta-lactam resistance:
>blaCTX-M-89_2_FJ971899 Beta-lactam resistance:
>blaCTX-M-90_1_FJ907381 Beta-lactam resistance:
>blaCTX-M-91_1_GQ870432 Beta-lactam resistance:
>blaCTX-M-92_1_GU127598 Beta-lactam resistance:
>blaCTX-M-93_1_HQ166709 Beta-lactam resistance:
>blaCTX-M-94_1_HM167760 Beta-lactam resistance:
>blaCTX-M-95_1_FN813245 Beta-lactam resistance:
>blaCTX-M-96_1_AJ704396 Beta-lactam resistance:
>blaCTX-M-97_1_HM776707 Beta-lactam resistance:Amino acid sequences of CTX-M-2 and CTX-M-97 are identical
>blaCTX-M-98_1_HM755448 Beta-lactam resistance:
>blaCTX-M-99_1_HM803271 Beta-lactam resistance:
>blaCTX-M-100_1_FR682582 Beta-lactam resistance:
>blaCTX-M-101_1_HQ398214 Beta-lactam resistance:
>blaCTX-M-102_1_HQ398215 Beta-lactam resistance:
>blaCTX-M-103_1_HG423149 Beta-lactam resistance:
>blaCTX-M-104_1_HQ833652 B

>blaOXA-240_1_JX089628 Beta-lactam resistance:
>blaOXA-241_1_JX025021 Beta-lactam resistance:
>blaOXA-242_1_JX025022 Beta-lactam resistance:
>blaOXA-243_1_JX206446 Beta-lactam resistance:
>blaOXA-244_1_JX438000 Beta-lactam resistance:
>blaOXA-245_1_JX438001 Beta-lactam resistance:
>blaOXA-247_1_JX893517 Beta-lactam resistance:
>blaOXA-248_1_HE963769 Beta-lactam resistance:
>blaOXA-249_1_HE963770 Beta-lactam resistance:
>blaOXA-250_1_HE963771 Beta-lactam resistance:
>blaOXA-251_1_JN118546 Beta-lactam resistance:
>blaOXA-253_1_KC479324 Beta-lactam resistance:
>blaOXA-254_1_AB781687 Beta-lactam resistance:
>blaOXA-255_1_KC479325 Beta-lactam resistance:
>blaOXA-256_1_HE616889 Beta-lactam resistance:
>blaOXA-257_1_KC567681 Beta-lactam resistance:
>blaOXA-258_1_HE614014 Beta-lactam resistance:
>blaOXA-278_1_KC771279 Beta-lactam resistance:
>blaOXA-309_1_HF947514 Beta-lactam resistance:
>blaOXA-320_1_KF151169 Beta-lactam resistance:
>blaOXA-322_1_KF203096 Beta-lactam resistance:
>blaOXA-323_1

>blaTEM-153_1_KC149518 Beta-lactam resistance:
>blaTEM-154_1_FJ807656 Beta-lactam resistance:Alternate name; CTM-type
>blaTEM-155_1_DQ679961 Beta-lactam resistance:
>blaTEM-156_1_AM941159 Beta-lactam resistance:
>blaTEM-157_1_DQ909059 Beta-lactam resistance:
>blaTEM-158_1_EF534736 Beta-lactam resistance:Alternate name; CTM-9
>blaTEM-159_1_EF136376 Beta-lactam resistance:
>blaTEM-160_1_EF136377 Beta-lactam resistance:
>blaTEM-162_1_EF468463 Beta-lactam resistance:
>blaTEM-163_1_EU815939 Beta-lactam resistance:
>blaTEM-164_1_EU274580 Beta-lactam resistance:
>blaTEM-166_1_FJ197316 Beta-lactam resistance:
>blaTEM-167_1_FJ360884 Beta-lactam resistance:
>blaTEM-168_1_FJ919776 Beta-lactam resistance:
>blaTEM-171_1_GQ149347 Beta-lactam resistance:
>blaTEM-176_1_GU550123 Beta-lactam resistance:
>blaTEM-177_1_FN652295 Beta-lactam resistance:
>blaTEM-178_1_X97254 Beta-lactam resistance:
>blaTEM-182_1_HQ317449 Beta-lactam resistance:
>blaTEM-183_1_HQ529916 Beta-lactam resistance:
>blaTEM-184_1_FR8

>fosX_3_CP003505 Fosfomycin resistance:
>fosX_4_DQ342344 Fosfomycin resistance:
>fosC_1_Z33413 Fosfomycin resistance:
>fosB_3_ACHE01000077 Fosfomycin resistance:
>fosB_4_CP000029 Fosfomycin resistance:
>fosB_5_AJ605334 Fosfomycin resistance:
>fosA_3_ACWO01000079 Fosfomycin resistance:
>fosA_4_ACWU01000146 Fosfomycin resistance:
>fosA_5_AFBO01000747 Fosfomycin resistance:
>fosB5_1_KT032253 Fosfomycin resistance:
>fosB6_1_KR870314 Fosfomycin resistance:
>fosB4_1_KR870311 Fosfomycin resistance:
>fosA_7_AEXB01000013 Fosfomycin resistance:
>fosA_2_AGDM01000012 Fosfomycin resistance:
>fosA_6_ACZD01000244 Fosfomycin resistance:
>fusB_1_AM292600 Fusidic acid resistance:
>fusB_3_JF777505 Fusidic acid resistance:
>VanR-A_1_FJ866609 Vancomycin resistance (Glycopeptid resistance):VanA vancomycin resistance operon, (VanA, VanH-A, VanR-A, VanS-A, VanX-A, VanY-A and VanZ-A)
>VanH-A_1_FJ866609 Vancomycin resistance (Glycopeptid resistance):VanA vancomycin resistance operon, (VanA, VanH-A, VanR-A, VanS

>qepA2_1_EU847537 Quinolone resistance:
>norA_1_M97169 Fluoroquinolone resistance:
>oqxA_1_EU370913 Quinolone resistance:
>oqxB_1_EU370913 Quinolone resistance:
>QnrB42_1_JN680743 Quinolone resistance:
>QnrB48_1_JQ762640 Quinolone resistance:
>QnrB49_1_JQ582718 Quinolone resistance:
>QnrB50_1_JX440357 Quinolone resistance:
>QnrB51_1_JX440358 Quinolone resistance:
>QnrB52_1_EF488762 Quinolone resistance:
>QnrB53_1_HQ704413 Quinolone resistance:
>QnrB54_1_HE820727 Quinolone resistance:
>QnrB56_1_JX259317 Quinolone resistance:
>QnrB57_1_JX259318 Quinolone resistance:
>QnrB58_1_JX259319 Quinolone resistance:
>QnrB59_1_JX259320 Quinolone resistance:
>QnrB60_1_AB734055 Quinolone resistance:
>QnrB61_1_AB734053 Quinolone resistance:
>QnrB62_1_JX987101 Quinolone resistance:
>QnrB64_1_KC580653 Quinolone resistance:
>QnrB65_1_KC580654 Quinolone resistance:
>QnrB66_1_KC580655 Quinolone resistance:
>QnrB67_1_KC580656 Quinolone resistance:
>QnrB68_1_KC580657 Quinolone resistance:
>QnrB69_1_KC580658 

In [10]:
RES_TEST  = ["ABC", "DEF", "GHI", "ABC"]
READ_LIST_2 = ["DEF", "GHI"]

# searching in a list (except when using index) is slower than
# searching in a dict or set. Here, generate a k-mer set, which
# is used to search the single read k-mer in.
# In the end, sequence must fully match, thus we can generate the
# k-mer from position 0 to k-mer_length (in our case 0 - 19)
def k_mer_set_of_RES(sequence, kmer_length):
    '''Return a set of k-mers of given sequence.
    We use it to search the single read k-mer in this set,
    since its faster than searching in a list (except when using index)'''
    k_mer_set = set()
    for i in range(0, len(sequence), 1):
        if (i < len(sequence)-kmer_length+1):
            k_mer_set.add(sequence[i:i+kmer_length])
    return(k_mer_set)

def reverse_complement(DNA_sequence):
    '''Reverse complement a given DNA sequence'''
    rev_comp_seq = ""
    trans_DNA = DNA_sequence.translate(DNA_translation_table)
    for char in range(len(trans_DNA)-1, -1, -1):
        rev_comp_seq += trans_DNA[char]
    return(rev_comp_seq)



# FOLLOWING FUNCTIONS ARE CURRENTLY NOT USED:
def kmer_generator(sequence, kmer_length):
    '''k-mer generator using list'''
    kmer_list = list()
    for i in range(0, len(sequence), 1):
        if (i < len(sequence)-kmer_length+1):
            kmer_list.append(sequence[i:i+kmer_length])
    return(kmer_list)

def get_counter_list(RES_LIST):
    '''returns a list in list. generates for each k-mer a counter'''
    counterlist = list()
    for entry in RES_LIST:
        counterlist.append([entry, 0])
    return(counterlist)
# counterlist = get_counter_list(RES_TEST) # print list with k-mer and its coverage

def incr_counter_list(RES_LIST, READ_LIST):
    '''increase the k-mer counter'''
    for pos in range(len(RES_LIST)):
        for read_kmer in READ_LIST:
            if read_kmer == RES_LIST[pos][0]: # in
                #print("DOM")
                RES_LIST[pos][1] += 1
        #print(RES_LIST[pos][0], RES_LIST[pos][1])
    #return(RES_LIST) # do not return to modify original list
# incr_counter_list(get_counter_list(RES_TEST), READ_LIST_2)

## Following just for testing purposes:

In [47]:
RES  = "ACGTACGTACGT" # Resistence gene
SEQ1 = "TTACGT"       # Contains last 3 resistance bases (so only part)
SEQ2 = "TACGTACGT"    # Entire sequence part of resistence gene
SEQ3 = "TACGTTA"      # Contains first 4 resistence bases (first part)
# Initialize list that keeps track of the depth
seq_list_count = list()

for i in range(len(RES)):
    seq_list_count.append(0)

SEQ1_kmer = kmer_generator(SEQ1, kmer_length)
SEQ2_kmer = kmer_generator(SEQ2, kmer_length)
SEQ3_kmer = kmer_generator(SEQ3, kmer_length)
RES_kmer  = kmer_generator(RES, kmer_length)

# check first SEQ k-mer (idea: COMPLETE SEQ-RES MATCH)
# for this method only the first k_mer of SEQ needs to be generated => shorter runtime
def first_kmer_check(SEQ_kmers, RES_kmers):
    for i in range(0, len(RES_kmers)-1, 1):
        if (SEQ_to_check[0] == RES_to_check[i]):
            print("Found match in first iteration. Investigating this SEQ sequence further.")
            return(True, i)
    print("No initial match. Discarding this SEQ.")
    return(False, 0)

SEQ_to_check = SEQ3_kmer
RES_to_check = RES_kmer

first_hit, start_pos = first_kmer_check(SEQ_to_check, RES_to_check)
end_pos = 0

i = 0
j = start_pos

length_SEQ = len(SEQ_to_check) - 1
length_RES = len(RES_to_check) - 1

is_mismatch = False

if (first_hit):
    while (i <= length_SEQ):
        while (j <= length_RES):
            print(i, SEQ_to_check[i], j, RES_to_check[j])
            if (SEQ_to_check[i] == RES_to_check[j]):
                print("Match")
                end_pos = j
                j += 1
                break
            elif (SEQ_to_check[i] != RES_to_check[j]):
                print("Mismatch")
                is_mismatch = True
                break
            j += 1
        if (is_mismatch):
            j = 0
            break
        i += 1

print("Start in RES:", start_pos, "End in RES:", start_pos+end_pos)

for i in range(start_pos, start_pos+end_pos, 1):
    seq_list_count[i] += 1

for i in range(len(RES)):
    print(RES[i], seq_list_count[i])

# RES:  ACGTACGTACGT
# SEQ2: ---TACGTTA--

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Found match in first iteration. Investigating this SEQ sequence further.
0 TAC 3 TAC
Match
1 ACG 4 ACG
Match
2 CGT 5 CGT
Match
3 GTT 6 GTA
Mismatch
Start in RES: 3 End in RES: 8
A 0
C 0
G 0
T 1
A 1
C 1
G 1
T 1
A 0
C 0
G 0
T 0
